In [1]:
import xarray as xr
import numpy as np
import xesmf as xe

In [2]:
def pre_func(nc):
    time = nc.time[0].values-np.timedelta64(1)
    time = time.astype('datetime64[D]').astype('datetime64[ns]')
    lead = np.arange(1,61,1)
    nc = nc.assign_coords({'lead':('time',lead)})
    nc = nc.swap_dims({'time':'lead'}).drop_vars('time')
    nc = nc.expand_dims({'time':[time]})
    return nc

def mask_hind(GShind_ncs,region):
    boxnames={'PM':[100,105,2.5,7.5],
        'EM':[110,120,2.5,7.5],
        'WI':[100,105,-10,2.5],
        'SI':[105,117,-10,-5],
        'NI':[107.5,118,-5,2.5],
        'EI':[118,125,-5,2.5],
        'SP':[120,130,5,12.5],
        'NP':[120,130,12.5,20],
        'SV':[104,110,8,11.5],
        'CV':[107,110,11.5,16.5],
        'NV':[103,108,17,23],
        'ST':[98,101,6.5,14],
        'NT':[99,105,14,19],
        'ING':[130,140,-10,0],
        'PNG':[140,150,-10,0]}[region]
    GS_mask_ncs = xr.open_mfdataset(f'/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6/*/*/GLOSEA6_lsmask_*.nc').isel(time=0)
    if 'time' in GS_mask_ncs:
        GS_mask_ncs = GS_mask_ncs.drop('time')

    # In[ ]:
    if region =='PM':
        mask_nc = xr.where(GS_mask_ncs.latitude>(-.85*(GS_mask_ncs.longitude-106)+4),0,GS_mask_ncs)
        mask_nc = xr.where(mask_nc.latitude<(-.85*(mask_nc.longitude-106)-1),0,mask_nc)
        mask_nc = xr.where(np.logical_or(mask_nc.longitude<97,mask_nc.longitude>106),0,mask_nc)
    elif region =='WI':
        mask_nc = xr.where(GS_mask_ncs.latitude>(-.85*(GS_mask_ncs.longitude-106)-1),100,GS_mask_ncs)
        mask_nc = xr.where(mask_nc.latitude<(-.85*(mask_nc.longitude-106)-7.5),100,mask_nc)
        mask_nc = xr.where(np.logical_or(mask_nc.longitude<95,mask_nc.longitude>106),100,mask_nc)
    else:
        mask_nc = xr.where(np.logical_or(GS_mask_ncs.longitude<boxnames[0],GS_mask_ncs.longitude>boxnames[1]),0,GS_mask_ncs)
        mask_nc = xr.where(np.logical_or(mask_nc.latitude<boxnames[2],mask_nc.latitude>boxnames[3]),0,mask_nc)
    print(mask_nc)
    print(GShind_ncs)
    regridder = xe.Regridder(GShind_ncs, mask_nc, 'conservative')
    GShind_ncs = regridder(GShind_ncs)
    return xr.where(mask_nc.lsm>.2,GShind_ncs.tp,np.nan)


In [21]:
#cf_nc = xr.open_mfdataset('/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6/2022/12/GLOSEA6_pr_*_cf.nc',preprocess=pre_func)
import glob
fnames = np.concatenate([glob.glob(f'/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6/2022/{i}/GLOSEA6_pr_*_cf.nc') for i in [11,12]])
cf_nc = xr.open_mfdataset(fnames,preprocess=pre_func)
fnames = np.concatenate([glob.glob(f'/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6/2022/{i}/GLOSEA6_pr_*_pf.nc') for i in [11,12]])
pf_nc = xr.open_mfdataset(fnames,preprocess=pre_func)

In [22]:
cf_nc = cf_nc.expand_dims({'number':[0]})

In [23]:
pr_nc = xr.concat([cf_nc,pf_nc],'number')
pr_nc = xr.concat([pr_nc.sel(lead=1),pr_nc.diff('lead')],'lead')


In [24]:
pr_nc

<xarray.Dataset> Size: 1GB
Dimensions:    (time: 61, longitude: 141, latitude: 71, lead: 60, number: 4)
Coordinates:
  * time       (time) datetime64[ns] 488B 2022-11-01 2022-11-02 ... 2022-12-31
  * longitude  (longitude) float32 564B 50.0 51.0 52.0 ... 188.0 189.0 190.0
  * latitude   (latitude) float32 284B 35.0 34.0 33.0 32.0 ... -33.0 -34.0 -35.0
  * lead       (lead) int64 480B 1 2 3 4 5 6 7 8 9 ... 53 54 55 56 57 58 59 60
  * number     (number) int64 32B 0 1 2 3
Data variables:
    tp         (number, time, latitude, longitude, lead) float64 1GB dask.array<chunksize=(1, 1, 71, 141, 1), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-01-03 12:38:54 GMT by grib_to_netcdf-2.32.0: /opt/ecmw...

In [25]:
masked_pr = mask_hind(pr_nc,'PM')

/tmp/ipykernel_12009/896763149.py:28: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  GS_mask_ncs = GS_mask_ncs.drop('time')


<xarray.Dataset> Size: 81kB
Dimensions:    (longitude: 141, latitude: 71)
Coordinates:
  * longitude  (longitude) float32 564B 50.0 51.0 52.0 ... 188.0 189.0 190.0
  * latitude   (latitude) float32 284B 35.0 34.0 33.0 32.0 ... -33.0 -34.0 -35.0
Data variables:
    lsm        (longitude, latitude) float64 80kB dask.array<chunksize=(141, 71), meta=np.ndarray>
<xarray.Dataset> Size: 1GB
Dimensions:    (time: 61, longitude: 141, latitude: 71, lead: 60, number: 4)
Coordinates:
  * time       (time) datetime64[ns] 488B 2022-11-01 2022-11-02 ... 2022-12-31
  * longitude  (longitude) float32 564B 50.0 51.0 52.0 ... 188.0 189.0 190.0
  * latitude   (latitude) float32 284B 35.0 34.0 33.0 32.0 ... -33.0 -34.0 -35.0
  * lead       (lead) int64 480B 1 2 3 4 5 6 7 8 9 ... 53 54 55 56 57 58 59 60
  * number     (number) int64 32B 0 1 2 3
Data variables:
    tp         (number, time, latitude, longitude, lead) float64 1GB dask.array<chunksize=(1, 1, 71, 141, 1), meta=np.ndarray>
Attributes:
    Conven

In [26]:
P95 = xr.open_dataset(f"/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6_hind/hind_P95_lead_allregions.nc")
area_thres = xr.open_dataarray(f"/gws/nopw/j04/ncas_climate_vol1/users/bc917929/GLOSEA6_hind/hind_areathres_pc10_DJF_lead_allregions.nc")

In [27]:
pr_bool = masked_pr>P95.sel(season='DJF',region='PM')
pr_bool = xr.where(np.isnan(masked_pr),np.nan,pr_bool.P95_lead).mean(['longitude','latitude'])

In [28]:
masked_pr

<xarray.DataArray (longitude: 141, latitude: 71, number: 4, time: 61, lead: 60)> Size: 1GB
dask.array<where, shape=(141, 71, 4, 61, 60), dtype=float64, chunksize=(141, 71, 3, 1, 59), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 564B 50.0 51.0 52.0 ... 188.0 189.0 190.0
  * latitude   (latitude) float32 284B 35.0 34.0 33.0 32.0 ... -33.0 -34.0 -35.0
  * time       (time) datetime64[ns] 488B 2022-11-01 2022-11-02 ... 2022-12-31
  * lead       (lead) int64 480B 1 2 3 4 5 6 7 8 9 ... 53 54 55 56 57 58 59 60
  * number     (number) int64 32B 0 1 2 3

In [29]:
area_thres

<xarray.DataArray (lead: 60, region: 15)> Size: 7kB
[900 values with dtype=float64]
Coordinates:
  * lead     (lead) int64 480B 1 2 3 4 5 6 7 8 9 ... 52 53 54 55 56 57 58 59 60
  * region   (region) <U3 180B 'PM' 'EM' 'WI' 'SI' ... 'ST' 'NT' 'ING' 'PNG'

In [30]:
pr_prob = (pr_bool>area_thres.sel(region='PM')).mean('number')

In [31]:
pr_prob.to_netcdf('./inputs/GS6/GS6_pr_event2.nc')

In [32]:
pr_prob


<xarray.DataArray (time: 61, lead: 60)> Size: 29kB
dask.array<mean_agg-aggregate, shape=(61, 60), dtype=float64, chunksize=(1, 59), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 488B 2022-11-01 2022-11-02 ... 2022-12-31
  * lead      (lead) int64 480B 1 2 3 4 5 6 7 8 9 ... 52 53 54 55 56 57 58 59 60
    region    <U3 12B 'PM'
    quantile  float64 8B ...
    season    <U3 12B 'DJF'